In [ ]:
import sys
# setting path
sys.path.append('../RSA')
# importing
import Shor_library
import random
from math import gcd
import math
from qiskit.circuit import Gate
from qiskit import ClassicalRegister, QuantumCircuit, QuantumRegister
from qiskit import Aer, execute, IBMQ, BasicAer, transpile
from qiskit.aqua import QuantumInstance

In [ ]:
def circuit_builder(N,a,sequential,method):
    n = math.ceil(math.log(N,2))
    if method=='Basic':
        aux = QuantumRegister(n+2)
        if sequential=='False':
            up_reg = QuantumRegister(2*n)
        else:
            up_reg = QuantumRegister(1)
            c_aux = ClassicalRegister(1)
        down_reg = QuantumRegister(n)
        up_classic = ClassicalRegister(2*n)
        if sequential=='False':
            circuit = QuantumCircuit(down_reg , up_reg , aux, up_classic)
        else:
            circuit = QuantumCircuit(down_reg , up_reg , aux, up_classic,c_aux)
        if sequential=='False':
            circuit.h(up_reg)
            circuit.x(down_reg[0])
            for i in range(0, 2*n):
                Shor_library.Ua_basic(circuit, up_reg[i], down_reg, aux, int(pow(a, pow(2, i))), N, n)
            Shor_library.create_inverse_QFT(circuit, up_reg, 2*n ,1)
            circuit.measure(up_reg,up_classic)
        else:
            circuit.x(down_reg[0])
            for i in range(0, 2*n):
                circuit.x(up_reg).c_if(c_aux, 1)
                circuit.h(up_reg)
                Shor_library.Ua_basic(circuit, up_reg[0], down_reg, aux, a**(2**(2*n-1-i)), N, n)
                for j in range(0, 2**i):
                    circuit.p(Shor_library.getAngle(j, i), up_reg[0]).c_if(up_classic, j)
                circuit.h(up_reg)
                circuit.measure(up_reg[0], up_classic[i])
                circuit.measure(up_reg[0], c_aux[0])
    else:
        if sequential=='False':
            aux = QuantumRegister(n+1)                          
            up_reg = QuantumRegister(2*n)
            down_reg = QuantumRegister(n)
            up_classic = ClassicalRegister(2*n)
            circuit = QuantumCircuit(down_reg , up_reg , aux, up_classic)
            circuit.h(up_reg)
            circuit.x(down_reg[0])
            for i in range(0, 2*n):
                Shor_library.Ua_inplace(circuit, up_reg[i:i+1], down_reg, aux, int(pow(a, pow(2, i))), N, n)
            Shor_library.create_inverse_QFT(circuit, up_reg, 2*n ,1)
            circuit.measure(up_reg,up_classic)
        else:
            aux = QuantumRegister(n+1)                          
            up_reg = QuantumRegister(1)
            down_reg = QuantumRegister(n)
            up_classic = ClassicalRegister(2*n)
            c_aux = ClassicalRegister(1)
            circuit = QuantumCircuit(down_reg , up_reg , aux, up_classic,c_aux)
            circuit.x(down_reg[0])
            for i in range(0, 2*n):
                circuit.x(up_reg).c_if(c_aux, 1)
                circuit.h(up_reg)
                Shor_library.Ua_inplace(circuit, up_reg[0:1], down_reg, aux, a**(2**(2*n-1-i)), N, n)
                for j in range(0, 2**i):
                    circuit.p(Shor_library.getAngle(j, i), up_reg[0]).c_if(up_classic, j)
                circuit.h(up_reg)
                circuit.measure(up_reg[0], up_classic[i])
                circuit.measure(up_reg[0], c_aux[0])
    return circuit

In [ ]:
def get_factors(counts,N,a):
    i=0
    prob_success=0
    while i < len(counts):
        """ Get the x_value from the final state qubits """
        output_desired = list(counts.keys())[i]
        #print(output_desired)
        #print(output_desired[1::])
        #output_desired=output_desired.split(" ")[1]
        x_value = int(output_desired[1::], 2)
        prob_this_result = 100 * ( int( list(counts.values())[i] ) ) / (100)
        """print("-----> Analysing result {0}. This result happened in {1:.4f} % of all cases\n".format(output_desired,prob_this_result))"""
        """ Print the final x_value to user """
        """print('In decimal, x_final value for this result is: {0}\n'.format(x_value))"""
        """ Get the factors using the x value obtained """   
        (p,q) = Shor_library.get_factors(int(x_value),int(N),int(a))
        """print('{0},{1}'.format(p,q))"""
        if p!=0 or q!=0:
            prob_success = prob_success + prob_this_result
            pstar=p
            qstar=q
        i=i+1
        P=prob_success
    print('p:',pstar,'q:',qstar,'P:',P)
    return P

In [ ]:
from pyquil import Program, get_qc
from pyquil.gates import X, CPHASE, CNOT, CSWAP, H, SWAP, MEASURE, PHASE
from functools import reduce
import numpy as np

from quantastica.qps_api import QPS

In [ ]:
QPS.save_account("<insert_your_token>","https://quantum-circuit.com/api")

In [ ]:
N=35
a=2
qc = circuit_builder(N,a,'True','Basic')
QASM_circ = qc.qasm()
#print(QASM_circ)
output_program = QPS.converter.convert(QASM_circ, "qasm", "pyquil")
PROB=[]
for i in range(0,1):
    """We must execute the first part of the program(creation of the circuit),
    but we must change the second part, that starts from len(output_program)-281."""
    exec(output_program[0:len(output_program)-281])
    shots = 100
    p.wrap_in_numshots_loop(shots)
    """For Sequential QFT version and basic method we must use a qvm of 2n+3 qubits.
    N=15 --> 11q-qvm
    N=20 --> 13q-qvm
    N=35 --> 15q-qvm"""
    qc = get_qc('15q-qvm',execution_timeout=2000)
    results_list = qc.run(p).readout_data.get("ro")
    results = list(map(lambda arr: reduce(lambda x, y: str(x) + str(y), arr[::-1], ""), results_list))
    counts = dict(zip(results,[results.count(i) for i in results]))
    P=get_factors(counts,N,a)
    PROB.append(P)
print(PROB)